In [1]:
import torch
import numpy as np

# Fixar a semente do Torch para operações específicas
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Importa e plota tensor

In [2]:
#First load libraries and images
%matplotlib inline
import math
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import numpy as np
import kornia
import kornia as K
import cv2
from kornia.feature import *
from time import time
import torch.optim as optim
from torch.nn import Parameter
from kornia.color import rgb_to_grayscale
import torchvision
from torchvision.transforms import transforms, InterpolationMode

from skimage import data

img_size =120
batch_size = 60
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((img_size, img_size), interpolation=InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform2 = transforms.Compose([
    transforms.Resize((img_size, img_size), interpolation=InterpolationMode.BICUBIC),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

trainset = torchvision.datasets.Flowers102(root='./data/datasets', split='train',
                                        download=True, transform=transform2)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

testset = torchvision.datasets.Flowers102(root='./data/datasets', split='test',
                                        download=True, transform=transform2)

num_datapoints_to_keep = math.ceil(len(testset) / 2)
num_datapoints_to_keep = 1020
indices_to_keep = torch.randperm(num_datapoints_to_keep)[:num_datapoints_to_keep]
reduced_testset = torch.utils.data.Subset(testset, indices_to_keep)
testloader = torch.utils.data.DataLoader(reduced_testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

## Aumento de dados caminho de ida e volta e reprodutibilidade

In [3]:
import kornia

class AugmentationParamsGenerator:
    def __init__(self, n, shape):
        torch.manual_seed(0)
        torch.cuda.manual_seed(0)
        
        aug_list = kornia.augmentation.AugmentationSequential(
            kornia.augmentation.RandomAffine(degrees=360, translate=(0.2, 0.2), scale=(0.95, 1.05), shear=10,p=0.8),
            kornia.augmentation.RandomPerspective(0.1, p=0.7),
            kornia.augmentation.RandomBoxBlur((5,5),p=0.7),
            # kornia.augmentation.RandomEqualize(p=0.3),
            data_keys=["input"],
            same_on_batch=True,
            # random_apply=10,
        )

        self.index = 0
        self.data = []
        for i in range(n):
            out = aug_list(torch.rand(shape))
            self.data.append(aug_list._params)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index >= len(self.data):
            self.index = 0  # Reset index to start over for circular iteration
            
        result = self.data[self.index]
        self.index += 1
        return result


def generate_random_points(image_size, num_points, min_distance=10):
    # Extrair as dimensões da imagem
    H, W = image_size

    # Inicializar a lista de pontos válidos
    valid_points = []

    while len(valid_points) < num_points:
        # Gerar um tensor com coordenadas aleatórias
        random_coords = torch.rand(1, 2)
        random_coords[:, 0] *= W
        random_coords[:, 1] *= H

        # Se não houver pontos válidos, adicionar o primeiro ponto gerado
        if not valid_points:
            valid_points.append(random_coords)
        else:
            # Verificar a distância entre o ponto gerado e os pontos válidos existentes
            distances = F.pairwise_distance(random_coords, torch.cat(valid_points, dim=0))
            min_distance_check = distances >= min_distance

            # Se o ponto estiver afastado o suficiente dos pontos válidos existentes, adicioná-lo
            if min_distance_check.all():
                valid_points.append(random_coords)

    # Concatenar todos os pontos válidos e retorná-los como um tensor
    return torch.cat(valid_points, dim=0)

In [4]:
import kornia
import matplotlib.pyplot as plt

def plot_patches_side_by_side(imgs_patches):
    num_imgs = imgs_patches.shape[0]  # Número de imagens
    fig, axs = plt.subplots(1, num_imgs, figsize=(num_imgs*4, 4))

    axs = axs.reshape((1, num_imgs))  # Ajustar a forma para matriz 2D com uma única linha

    for i in range(num_imgs):
        axs[0, i].imshow(kornia.tensor_to_image(imgs_patches[i]))
        axs[0, i].axis('off')

    plt.show()
    
def plot_images_with_points_side_by_side(image1, image2, points1=None, points2=None):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # Cria uma figura com duas subplots

    # Converte tensores para imagens
    image1 = kornia.utils.tensor_to_image(image1)
    image2 = kornia.utils.tensor_to_image(image2)

    # Plot da primeira imagem na subplot da esquerda
    axs[0].imshow(image1)
    if points1 is not None:
        points1 = points1.cpu().numpy()  # Converte para numpy
        keypoints_x = points1[:,0].flatten().tolist()
        keypoints_y = points1[:,1].flatten().tolist()
        axs[0].scatter(keypoints_x, keypoints_y, c='red', marker='x')  # Plota os pontos em vermelho com marcador 'x'
    axs[0].axis('off')  # Remove os eixos

    # Plot da segunda imagem na subplot da direita
    axs[1].imshow(image2)
    if points2 is not None:
        points2 = points2.cpu().numpy()  # Converte para numpy
        keypoints_x = points2[:,0].flatten().tolist()
        keypoints_y = points2[:,1].flatten().tolist()
        axs[1].scatter(keypoints_x, keypoints_y, c='red', marker='x')
    axs[1].axis('off')  # Remove os eixos

    plt.show()  # Mostra o plot com as duas imagens lado a lado

def plot_images_with_points_side_by_side(image1, image2, points1=None, points2=None):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # Cria uma figura com duas subplots

    # Converte tensores para imagens
    image1 = kornia.utils.tensor_to_image(image1)
    image2 = kornia.utils.tensor_to_image(image2)

    # Plot da primeira imagem na subplot da esquerda
    axs[0].imshow(image1)
    if points1 is not None:
        points1 = points1.cpu().numpy()  # Converte para numpy
        keypoints_x = points1[:,0].flatten()
        keypoints_y = points1[:,1].flatten()
        axs[0].scatter(keypoints_x, keypoints_y, c='red', marker='x')  # Plota os pontos em vermelho com marcador 'x'
        
        # Adiciona os números dos labels aos pontos na subplot da esquerda
        for i, (x, y) in enumerate(zip(keypoints_x, keypoints_y)):
            axs[0].text(x, y, str(i), color='red')

    axs[0].axis('off')  # Remove os eixos

    # Plot da segunda imagem na subplot da direita
    axs[1].imshow(image2)
    if points2 is not None:
        points2 = points2.cpu().numpy()  # Converte para numpy
        keypoints_x = points2[:,0].flatten()
        keypoints_y = points2[:,1].flatten()
        axs[1].scatter(keypoints_x, keypoints_y, c='red', marker='x')
        
        # Adiciona os números dos labels aos pontos na subplot da direita
        for i, (x, y) in enumerate(zip(keypoints_x, keypoints_y)):
            axs[1].text(x, y, str(i), color='red')

    axs[1].axis('off')  # Remove os eixos

    plt.show()  # Mostra o plot com as duas imagens lado a lado


def filtrar_keypoints(lista_de_pontos, tensor_mascara):
    # Verificar se as coordenadas estão dentro das dimensões
    dimensao_max_x, dimensao_max_y = tensor_mascara.shape[1] - 1, tensor_mascara.shape[0] - 1
    pontos_filtrados = [
        1  if 0 <= p1[0] <= dimensao_max_x 
        and 0 <= p1[1] <= dimensao_max_y 
        and tensor_mascara[int(p1[1]), int(p1[0])] else 0 for p1 in lista_de_pontos
    ]
    pontos_filtrados = torch.tensor(pontos_filtrados, dtype=torch.bool)
    lista_de_pontos = torch.tensor(lista_de_pontos)
    return lista_de_pontos[pontos_filtrados],pontos_filtrados

In [5]:
import kornia


iterator=iter(testloader)
input,labels = next(iterator)

B,C,H,W = input.shape
mask = torch.ones(B,C,H,W)
border_size =20
mask[:, :, :border_size, :] = 0
mask[:, :, -border_size:, :] = 0
mask[:, :, :, :border_size] = 0
mask[:, :, :, -border_size:] = 0

bbox = torch.tensor([[
    [1., 1.],
    [2., 1.],
    [2., 2.],
    [1., 2.],
]]).expand(B, 1, -1, -1)

points = generate_random_points((H,W),50).expand(B, -1, -1)
shape = input.shape
params_lists =AugmentationParamsGenerator(6,shape)
next_item = next(params_lists)
points = points.to(device)
mask = mask.to(device)
bbox = bbox.to(device)


In [6]:
from kornia.feature import laf_from_center_scale_ori
from kornia_moons.feature import visualize_LAF

def convert_points_to_lafs(points,img1, PS=19):
    orient = kornia.feature.LAFOrienter(PS)#
    scale_lafs = torch.ones(img1.shape[0],points.shape[1],1,1)*PS
    scale_lafs = scale_lafs.to(points.device)
    lafs1 = laf_from_center_scale_ori(points,scale_lafs)
    lafs2 = orient(lafs1, img1)
    return lafs2
    
def extract_patches_simple(batch, lafs, PS=19):
    # visualize_LAF(img1,lafs2,img_idx=0,figsize=(8,6))
    imgs_patches = kornia.feature.extract_patches_from_pyramid(batch, lafs, PS)
    # plot_patches_side_by_side(imgs_patches[0][0])#plota todas as features do patch 0 imagem 0
    return imgs_patches
    
def extract_patches_from_keypoints(batch,filtered_points, PS=13):
    '''
    Extrai patches das imagens com base nos pontos de interesse filtrados.

    Parâmetros:
        batch (torch.Tensor): Tensor contendo as imagens com o formato B, C, H, W. <=out[0]
        pontos (torch.Tensor): Tensor contendo os pontos de interesse com o formato B, N, 2. <=out[3][0]
        mask (torch.Tensor): Tensor binário indicando os pontos a serem mantidos, com o formato B, N. <=out[1][0, 0]
        PS (int): Tamanho do patch a ser extraído.

    Retorna:
        patchs_mini_img (torch.Tensor): Tensor contendo os patches extraídos com o formato B, N_filtered, C, PS, PS.
    '''
    CH = 8  # TODO temporário enquanto não aplico a convolução
    B, _, H, W = batch.shape
    filtered_points = filtered_points.repeat(B, 1, 1)  # B, N(filtered), 2
    assert torch.allclose(filtered_points[0], filtered_points[1]), 'pontos devem ser iguais em todas as imagens'
    lafs = convert_points_to_lafs(filtered_points, batch)  # B, N(filtered), 2, 3

    maps_activations = batch.repeat(1, CH, 1, 1)  # B, CH, H, W o repeat 8 é temporário enquanto não aplico a convolução
    # maps_activations = extract_features(model_sp,batch)
    print('maps_activations ',maps_activations.shape,maps_activations.device)
    # maps_activations[:, 2] = maps_activations[2, 2]
    patchs_mini_img = extract_patches_simple(maps_activations, lafs, PS)
    return patchs_mini_img


In [7]:
from e2cnn import gspaces
from e2cnn import nn as enn    #the equivariant layer we need to build the model
from torch import nn
class Feature(nn.Module):
    def __init__(self,n_channel=2) -> None:
        super().__init__()
        r2_act = gspaces.Rot2dOnR2(N=18)      

        feat_type_in  = enn.FieldType(r2_act,  n_channel*[r2_act.trivial_repr])
        feat_type_out = enn.FieldType(r2_act, 2*n_channel*[r2_act.regular_repr])     
        self.input_type = feat_type_in

        self.block1 = enn.SequentialModule(                
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=3, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True)
                )
        
        self.pool1 = enn.PointwiseAvgPoolAntialiased(feat_type_out, sigma=0.66, stride=1, padding=0)

        feat_type_in  = self.block1.out_type
        feat_type_out = enn.FieldType(r2_act,  4*n_channel*[r2_act.regular_repr])
        self.block2 = enn.SequentialModule(                
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=3, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True),
                )
        # self.pool2 = enn.PointwiseAvgPool(feat_type_out, 21)
        
        feat_type_in  = feat_type_out
        feat_type_out = enn.FieldType(r2_act,  8*n_channel*[r2_act.regular_repr])
        self.block3 = enn.SequentialModule(                
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=3, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True),
                )
        
        feat_type_in  = feat_type_out
        feat_type_out = enn.FieldType(r2_act,  16*n_channel*[r2_act.regular_repr])
        self.block4 = enn.SequentialModule(                
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=3, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True),
                enn.GroupPooling(feat_type_out)
                )
        self.pool = enn.PointwiseAdaptiveAvgPool(self.block4.out_type,1)
                
    def forward(self,X1)->torch.Tensor:
        x = enn.GeometricTensor(X1, self.input_type)
        n_dim = X1.shape[-1]
        mask = enn.MaskModule(self.input_type, n_dim, margin=2).to(X1.device)
        x = mask(x)     
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.pool(x)
        return x.tensor
    

class Discriminator(nn.Module):
    def __init__(self, n_classes=10) -> None:
        super().__init__()
        #criar camadas densa a partir de x que é uma cnn
        self.dense1 = nn.Linear(2*1*128, 256)
        self.dense2 = nn.Linear(256, 128)
        self.droupout = nn.Dropout(0.2)
        self.activation = nn.LeakyReLU()
        #self.activation = nn.ELU()
        #função de ativação ideal para retornar um valor entre 0 e 1
        self.activation2 = nn.Tanh()

    
    def forward(self,X1,X2)->torch.Tensor:
        flatten_x1 = X1.view(X1.size(0), -1)
        flatten_x2 = X2.view(X2.size(0), -1)
        x = torch.cat((flatten_x1,flatten_x2),dim=1)
        x = self.droupout(self.dense1(x))
        x = self.activation(x)
        x = self.droupout(self.dense2(x))
        x = self.activation(x)
        
        # Calculando a distância euclidiana
        x = torch.norm(x, dim=1)
        # x = self.activation(distance) #retorna um valor entre 0 e 1
        return x
    
class Siamesa(nn.Module):
    def __init__(self,n_channel=2) -> None:
            super().__init__()
            self.feature = Feature(n_channel=n_channel)
            self.discriminator = Discriminator()
    
    def forward(self,X1,X2)->torch.Tensor:
        x1 = self.feature(X1)
        x2 = self.feature(X2)
        x = self.discriminator(x1,x2)
        return x


n_channel =8
PS =21
model =Siamesa(n_channel=n_channel).to(device)
X1=torch.rand(50,n_channel,PS,PS).to(device)
X2=torch.rand(50,n_channel,PS,PS).to(device)
dist = model(X1,X1)
dist

/home/wagner/.local/lib/python3.11/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)


tensor([3.7678, 4.4151, 3.9970, 3.5622, 4.5677, 4.1504, 3.8267, 3.7374, 4.1247,
        3.4617, 4.1508, 3.5717, 3.7994, 4.1814, 4.1044, 3.8746, 3.6095, 4.4004,
        3.7782, 4.3038, 3.8337, 4.1391, 4.2422, 3.8081, 3.6655, 3.7796, 4.1400,
        4.1180, 3.9309, 3.9568, 4.3090, 4.6009, 3.6586, 4.3789, 3.5193, 4.4851,
        3.9907, 4.0311, 4.2564, 3.8601, 3.9827, 3.7911, 3.6861, 3.8840, 4.2896,
        3.5813, 3.6717, 4.5272, 3.7939, 4.0212], device='cuda:0',
       grad_fn=<LinalgVectorNormBackward0>)

In [8]:
from torch.nn.functional import pairwise_distance
from tqdm import tqdm
#Create methods to calculate loss
def loss_fn(output_pos,output_neg):  
      
    margim = 0.8
    zero = torch.tensor(0.,requires_grad=True).to(output_pos.device)
    loss = output_pos - output_neg + margim
    loss = torch.max(torch.tensor(0),loss)
    loss = torch.sum(loss)  # Reduzir para um escalar
    return loss

#Create methods to train the model
def train_one_epoch(model, data_loader, optimizer, loss_fn, device='cpu', is_training=True):
    model.train(is_training)
    total_loss = 0.
    # Definir os intervalos de colunas
    
    progress_bar = tqdm(data_loader)
    for data in progress_bar:
        #extrair as features e orientações
        batch_in,batch_out = data[0].to(device),data[1].to(device)
        #predição no cenário positivo                
        output_pos = model(batch_in, batch_out)
        #predição no cenário negativo
        batch_out_neg = torch.roll(batch_out, 1, 0)
        output_neg = model(batch_in, batch_out_neg)
        #calcular a loss
        loss = loss_fn(output_pos, output_neg)
        
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_description(f'Loss: {loss.item()} - Total Loss: {total_loss}')

    return total_loss / len(data_loader.dataset)


In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

class MeuDataset(Dataset):
    def __init__(self, summary_pool_list, labels_list):
        self.summary = summary_pool_list
        self.labels = labels_list
        
    def __len__(self):
        return len(self.summary )
    
    def __getitem__(self, idx):
        summary = self.summary[idx]
        labels = self.labels[idx]
        # Implemente aqui a lógica para retornar uma amostra do seu conjunto de dados
        return summary, labels

In [10]:
aug_list = kornia.augmentation.AugmentationSequential(
    kornia.augmentation.RandomAffine(degrees=360, translate=(0.2, 0.2), scale=(0.95, 1.05), shear=10,p=0.8),
    kornia.augmentation.RandomPerspective(0.1, p=0.7),
    kornia.augmentation.RandomBoxBlur((5,5),p=0.7),
    data_keys=["input", "mask", "bbox", "keypoints"],
    same_on_batch=True,
)

PS =23
print("dt aug",input.shape, mask.shape, bbox.shape, points.shape)
orient = PassLAF()#kornia.feature.LAFOrienter(PS)PassLAF()
patchs_mini_src = torch.tensor([])
patchs_mini_dst = torch.tensor([])
for input,labels in trainloader:
    input = input.to(device)
    with torch.no_grad():
        params_item = next(params_lists)
        out = aug_list(input, mask, bbox, points,params=params_item)
        #extrair patch da imagem original
        filtered_points,indices = filtrar_keypoints(out[3][0],out[1][0,0].bool())  
        print(input.shape,input.device,points.device,filtered_points.device)      
        patchs_mini_img1 =extract_patches_from_keypoints(input, points[0][indices],PS=PS)
        patchs_mini_img1 =patchs_mini_img1.reshape(-1,patchs_mini_img1.shape[2],PS,PS)
        patchs_mini_src = torch.cat((patchs_mini_src,patchs_mini_img1),dim=0)
        print(patchs_mini_img1.shape,patchs_mini_src.shape)
        plot_patches_side_by_side(patchs_mini_img1[:50,0])#plota todas os patch na feature map 0    
        #extrair patch da imagem transformada
        patchs_mini_img2 =extract_patches_from_keypoints(out[0], filtered_points,PS=PS)
        patchs_mini_img2 =patchs_mini_img2.reshape(-1,patchs_mini_img2.shape[2],PS,PS)
        patchs_mini_dst = torch.cat((patchs_mini_dst,patchs_mini_img2),dim=0)
        print(patchs_mini_img2.shape,patchs_mini_dst.shape)
        plot_patches_side_by_side(patchs_mini_img2[:50,0])#plota todas os patch na feature map 0    

        plot_images_with_points_side_by_side(input[0],out[0][0],points[0][indices],filtered_points)

dt aug torch.Size([60, 1, 120, 120]) torch.Size([60, 1, 120, 120]) torch.Size([60, 1, 4, 2]) torch.Size([60, 50, 2])


/tmp/ipykernel_31989/732539104.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lista_de_pontos = torch.tensor(lista_de_pontos)


torch.Size([60, 1, 120, 120]) cuda:0 cuda:0 cuda:0


NameError: name 'extract_features' is not defined

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
from torch.utils.data import random_split, DataLoader
print(patchs_mini_src.shape, patchs_mini_dst.shape)
optimizer_siamese = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
# Crie uma instância do seu conjunto de dados personalizado

meu_dataset = MeuDataset(patchs_mini_src, patchs_mini_dst)
train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.3,0.5,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=50, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=50, shuffle=True)



In [ ]:
patchs_mini_src.shape,len(dataloader.dataset)

(torch.Size([21240, 8, 23, 23]), 1260)